In [1]:
import os
import requests
import pandas as pd
import json

from pathlib import Path
import csv

from dotenv import load_dotenv
load_dotenv()

import pandas_datareader as web
import datetime as dt

%matplotlib inline

In [2]:
start = dt.datetime(2014,8,1)
end = dt.datetime.now()

btc_date = dt.datetime(2014,8,1)

iex_api_key = os.getenv("IEX_API_KEY")
iex_test_api_key = os.getenv("IEX_TEST_API_KEY")

base_url = 'https://cloud.iexapis.com/stable/'
sandbox_url = 'https://sandbox.iexapis.com/stable/'


token = os.environ.get('IEX_API_KEY')
test_token = os.environ.get('IEX_TEST_API_KEY')
params = {'token': token}
test_params = {'token': test_token}

test_resp = requests.get(base_url + 'status')
test_resp

<Response [200]>

In [3]:
def get_close(ticker):
    data = web.DataReader(ticker, 'yahoo', start, end)
    df = data[["Close"]]
    df.rename(columns={'Close':ticker},inplace=True)
    return df

def yahoo_10_year_data(tickers_list):
#     start = dt.datetime(2011,9,20)
#     end = dt.datetime.now()
    combined_df_f = pd.DataFrame()
    
    for i in tickers_list:
        if i == 0: 
            raw_df_1 = web.DataReader(i, 'yahoo', start, end)
            combined_df_f = raw_df_1[['Close']]
            combined_df_f.rename(columns={'Close':i},inplace=True)
            
        else:
            raw_df_2 = web.DataReader(i, 'yahoo', start, end)
            df_f = raw_df_2[['Close']]
            df_f.rename(columns={'Close':i},inplace=True)
            
            combined_df_f = pd.concat([combined_df_f,df_f], axis='columns')
            #combined_df_f = combined_df_f.merge(df_f, how='inner',right_index=True, left_index=True)


    return combined_df_f

def test_iex_apicall_get_close_data_73m(tickers_list):
    combined_df_f = pd.DataFrame()
    
    for i in tickers_list: 
        
        if i == 0: 
            resp_data = requests.get(sandbox_url+'stock/'+i+'/chart/75m?token='+test_token)
            combined_df_f = pd.DataFrame(resp_data.json())
            
            combined_df_f.rename(columns={'date':'Date'},inplace=True)
            combined_df_f['Date'] = pd.to_datetime(combined_df_f['Date'])
            combined_df_f = combined_df_f.set_index('Date')
            combined_df_f = combined_df_f[['close']]
            symbol = i 
            combined_df_f.rename(columns={'close':symbol},inplace=True)
            
        else: 
            resp_data = requests.get(sandbox_url+'stock/'+i+'/chart/75m?token='+test_token)
            resp_df = pd.DataFrame(resp_data.json())
            
            resp_df.rename(columns={'date':'Date'},inplace=True)
            resp_df['Date'] = pd.to_datetime(resp_df['Date'])
            resp_df = resp_df.set_index('Date')
            resp_df = resp_df[['close']]
            #symbol = data.iloc[0,4]
            symbol = i
            resp_df.rename(columns={'close':symbol},inplace=True)
            
            combined_df_f = pd.concat([combined_df_f,resp_df], axis='columns')
            
    return combined_df_f 

In [4]:
btc = get_close('BTC-USD')
eth = get_close('ETH-USD')
stock_list = ['GME','AMC','AMZN','WMT','NVDA','AMD','AAPL','MSFT','DIS','PYPL']
iex_stock_list = test_iex_apicall_get_close_data_73m(stock_list)
#iex_stock_list

C:\Users\watso\anaconda3\envs\pyvizenv\lib\site-packages\pandas\core\frame.py:4449: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,
C:\Users\watso\anaconda3\envs\pyvizenv\lib\site-packages\pandas\core\frame.py:4449: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [5]:
crypto_df = btc.merge(eth, how='inner',right_index=True, left_index=True)
#crypto_df

In [6]:
full_df = crypto_df.merge(iex_stock_list, how='inner',right_index=True, left_index=True)
#full_df

In [7]:
#full_df.plot(figsize=(10,5))

In [8]:
full_pct_change = full_df.pct_change()
#full_pct_change.plot(figsize=(10,5))

In [9]:
# Set the file path
csvpath = Path('Resources/1810020501_databaseLoadingData.csv')
csv_data = pd.read_csv(csvpath)

In [10]:
land_and_house = csv_data[["REF_DATE","New housing price indexes","VALUE"]]
land_and_house['NEW_REF_DATE'] = pd.to_datetime((land_and_house['REF_DATE'].astype(str) + "-01"))
land_and_house.rename(columns={'NEW_REF_DATE':'Date'},inplace=True)
land_and_house.set_index("New housing price indexes",inplace=True)

total_land_and_house_value = land_and_house.loc["Total (house and land)"]
total_land_and_house_value.set_index("Date",inplace=True)
total_land_and_house_value.rename(columns={'VALUE':"Total land+house Value"},inplace=True)
total_land_and_house_value = total_land_and_house_value[['Total land+house Value']]

land_value = land_and_house.loc["Land only"]
land_value.set_index("Date",inplace=True)
land_value.rename(columns={'VALUE':"Land Value"},inplace=True)
land_value = land_value[['Land Value']]

house_value = land_and_house.loc["House only"]
house_value.set_index("Date",inplace=True)
house_value.rename(columns={'VALUE':"House Value"},inplace=True)
house_value = house_value[['House Value']]

C:\Users\watso\anaconda3\envs\pyvizenv\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [11]:
combined_df = pd.concat([total_land_and_house_value,house_value,land_value], axis='columns')
#combined_df

In [12]:
only_house_values = combined_df[['House Value']]
#only_house_values

In [13]:
monthly_full_df = only_house_values.merge(full_df, how='inner',right_index=True, left_index=True)
# monthly_full_df.head()

In [14]:
#monthly_full_df.plot(figsize=(18,5))

In [15]:
monthly_change = monthly_full_df.pct_change()

In [16]:
monthly_change.dropna(inplace=True)

In [17]:
#monthly_change.plot()

In [18]:
# cum_monthly_change = monthly_change.cumprod()
# cum_monthly_change.tail()

cumulative_monthly_returns = (1+monthly_change).cumprod() - 1

In [19]:
# Calculate cumulative returns of all portfolios
#cumulative_returns_all = (1 + combined_df).cumprod() - 1

readable_print_out = cumulative_monthly_returns * 100
readable_print_out = round(readable_print_out, 2)

# print(readable_print_out.iloc[-1],'\n')
# # Plot cumulative returns
# cumulative_monthly_returns.plot(figsize=(18,5))

In [20]:
monthly_houseandstock_df = only_house_values.merge(iex_stock_list, how='inner',right_index=True, left_index=True)
# monthly_houseandstock_df

In [21]:
monthly_houseandstock_change = monthly_houseandstock_df.pct_change()
monthly_houseandstock_change.dropna(inplace=True)
cumulative_houseandstock_change = (1+monthly_houseandstock_change).cumprod() - 1

In [22]:
readable_print_out_2 = cumulative_houseandstock_change * 100
readable_print_out_2 = round(readable_print_out_2, 2)

# print(readable_print_out_2.iloc[-1],'\n')
# # Plot cumulative returns
# cumulative_houseandstock_change.plot(figsize=(18,5))

In [23]:
US_market_index = get_close('VUS.TO')

C:\Users\watso\anaconda3\envs\pyvizenv\lib\site-packages\pandas\core\frame.py:4449: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [24]:
# US_market_index

In [25]:
monthly_full_df_VS = US_market_index.merge(monthly_full_df, how='inner',right_index=True, left_index=True)
# monthly_full_df_VS.head()

In [26]:
# List of DataFrames  
# iex_stock_list      just stock close daily data 
# crypto_df           just crypto close daily data 
# full_df             iex and crypto combined 
# monthly_full_df     full_df combined with monthly housing values 
# monthly_houseandstock_df       only stocks and housing values 
# monthly_full_df_VS             monthly_full_df combined with VUS.TO index 
# combined_df                    combined_df of all housing price data 

In [27]:
iex_stock_list.to_csv('Resources/iex_stock_list.csv')
crypto_df.to_csv('Resources/crypto_df.csv')
full_df.to_csv('Resources/full_df.csv')
monthly_full_df.to_csv('Resources/monthly_full_df.csv')
monthly_houseandstock_df.to_csv('Resources/monthly_houseandstock_df.csv')
monthly_full_df_VS.to_csv('Resources/monthly_full_df_VS.csv')
combined_df.to_csv('Resources/combined_df.csv')

In [28]:
# plot_1 = monthly_full_df_VS.plot(y="VUS.TO")
# plot_2 = monthly_full_df_VS.plot(y="House Value")

In [29]:
iex_stock_df = pd.read_csv(
    Path("Resources/iex_stock_list.csv"),
    infer_datetime_format=True,
    parse_dates=True,
    #index_col="Date",
)
iex_stock_df_date_index = pd.read_csv(
    Path("Resources/iex_stock_list.csv"),
    infer_datetime_format=True,
    parse_dates=True,
    index_col="Date",
)
house_price = pd.read_csv(
    Path("Resources/combined_df.csv"),
    infer_datetime_format=True,
    parse_dates=True,
    #index_col="Date",
)
house_value = house_price[["Date","House Value"]]
crypto_df = pd.read_csv(
    Path("Resources/crypto_df.csv"),
    infer_datetime_format=True,
    parse_dates=True,
    index_col="Date",
)
vus_df = get_close("VUS.TO")
iex_stock_df_date_index = vus_df.merge(iex_stock_df_date_index, how='inner',right_index=True, left_index=True)
full_df = crypto_df.merge(iex_stock_df_date_index, how='inner',right_index=True, left_index=True)
full_df.reset_index(inplace=True)
new_df = full_df.iloc[::21,:]
new_df.reset_index(inplace=True)
new_df.drop(columns={"index","Date"},inplace=True)
final_df = pd.concat([house_value,new_df], axis="columns")
final_df.set_index("Date",inplace=True)
final_df.to_csv('Resources/final_df.csv')

C:\Users\watso\anaconda3\envs\pyvizenv\lib\site-packages\pandas\core\frame.py:4449: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,
C:\Users\watso\anaconda3\envs\pyvizenv\lib\site-packages\pandas\core\frame.py:4315: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [30]:
final_df

,House Value,BTC-USD,ETH-USD,VUS.TO,GME,AMC,AMZN,WMT,NVDA,AMD,AAPL,MSFT,DIS,PYPL
Date,,,,,,,,,,,,,,
2015-08-01,91.8,279.584991,2.772120,42.209999,46.06,30.76,549.41,73.88,5.2210,2.160,29.4109,47.45,110.89,40.06
2015-09-01,92.0,235.018997,1.338810,38.959999,43.24,28.95,516.00,65.70,5.4940,1.840,28.2082,43.44,105.74,35.23
2015-10-01,92.4,242.968994,0.609388,39.790001,44.27,26.29,553.29,66.81,6.6658,1.890,28.8143,47.77,104.89,33.31
2015-11-01,92.6,386.354004,0.895637,42.049999,47.77,27.53,668.77,60.85,7.0561,2.200,31.0200,56.53,116.00,37.24
2015-12-01,92.8,395.536011,0.811264,41.610001,33.15,23.68,687.43,62.60,8.4726,2.402,30.6200,55.82,116.40,36.37
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-04-01,118.3,37304.691406,2430.621338,78.820000,178.37,13.11,3279.31,147.29,152.0760,79.770,127.3300,249.94,175.37,261.98
2021-05-01,119.9,33723.027344,1989.736328,80.980003,222.90,61.14,3620.84,143.09,192.1138,85.580,139.2500,278.18,175.20,289.09
2021-06-01,120.7,34292.445312,2189.218750,83.430000,186.50,37.76,3786.43,144.35,199.9700,92.530,150.7500,304.15,179.91,311.55
